# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [135]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [136]:
# Path csv, read in csv, display csv
path = "../Output/city_data.csv"
csv = pd.read_csv(path)
csv.head()

,Unnamed: 0,City,Lat,Lng,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,0,Shakhtinsk,49.71,72.59,33.80,43,33,8.95,KZ,1603125274
1,1,Khatanga,71.97,102.50,9.48,95,38,4.88,RU,1603125274
2,2,Mil'kovo,54.72,158.62,33.13,91,88,5.26,RU,1603125274
3,3,Bambous Virieux,-20.34,57.76,75.20,73,40,8.05,MU,1603125159
4,4,Adrar,20.50,-10.07,92.89,13,0,14.97,MR,1603125275


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [137]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [138]:
# Use lat and lng as location 
locations = csv[["Lat", "Lng"]].astype(float)
weight_humidity = csv["Humidity (%)"].astype(float)

In [139]:
# Create gmaps.figure
fig = gmaps.figure()

# Creeate heat_layer with lat and lng as locations and humidity 
heat_layer = gmaps.heatmap_layer(locations, weights=weight_humidity,
                                dissipating=False, max_intensity=100,
                                point_radius = 1)

fig.add_layer(heat_layer)

# Save heatmap
plt.savefig("../Images/vacay_heatmap.png")

# Display
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [141]:
# Use loc to get:
# max temp 80 < and 70 >
# wind speed 10 <
# cloudiness = 0
# Create new dataframe with ideal conditions
hotel_df = csv.loc[(csv["Max Temp. (F)"] < 80) & 
                    (csv["Max Temp. (F)"] > 70) &
                    (csv["Wind Speed (mph)"] < 10) &
                    (csv["Cloudiness (%)"] == 0), :]
hotel_df = hotel_df.dropna()
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
69,69,Guerrero Negro,27.98,-114.06,79.20,56,0,5.35,MX,1603125291
109,109,Algiers,36.75,3.04,75.20,47,0,9.17,DZ,1603125187
110,110,Zambezi,-13.54,23.10,79.68,16,0,5.77,ZM,1603125300
147,147,Nisibis,37.08,41.22,78.80,24,0,6.93,TR,1603125308
167,167,Richards Bay,-28.78,32.04,73.99,81,0,9.60,ZA,1603125313
290,290,Tiznit Province,29.58,-9.50,79.47,24,0,2.06,MA,1603125342
321,321,Quelimane,-17.88,36.89,74.64,80,0,9.13,MZ,1603125348
414,414,Mīnāb,27.15,57.08,71.60,26,0,4.70,IR,1603125372
434,434,Bosaso,11.28,49.18,79.47,59,0,2.35,SO,1603125378


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [142]:
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
69,69,Guerrero Negro,27.98,-114.06,79.20,56,0,5.35,MX,1603125291,
109,109,Algiers,36.75,3.04,75.20,47,0,9.17,DZ,1603125187,
110,110,Zambezi,-13.54,23.10,79.68,16,0,5.77,ZM,1603125300,
147,147,Nisibis,37.08,41.22,78.80,24,0,6.93,TR,1603125308,
167,167,Richards Bay,-28.78,32.04,73.99,81,0,9.60,ZA,1603125313,


In [143]:
# URl 
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# Set geocoordinates
target_keyword = "hotel"
target_type = "hotels"
target_radius = 5000


# Set params
params = {
    "keyword": target_keyword,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}


# for loop to iterate through hotel_df 
for index, row in hotel_df.iterrows(): 
    
    # Set variable from lat and lng column to iterrate through
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add locations to params with lat and lng data that was collected
    params["location"] = f"{lat},{lng}"
    
    # Get response from API 
    response = requests.get(url, params=params).json()
    
    # Wrap for loop in try statement so that we can use except statement for any city that has missing data
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/results...skipping ")


Missing field/results...skipping 


In [147]:
# Display hotel_df 
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp. (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
69,69,Guerrero Negro,27.98,-114.06,79.20,56,0,5.35,MX,1603125291,Hotel Los Caracoles
109,109,Algiers,36.75,3.04,75.20,47,0,9.17,DZ,1603125187,Sofitel Algiers Hamma Garden
110,110,Zambezi,-13.54,23.10,79.68,16,0,5.77,ZM,1603125300,Royal Kutachika Lodge
147,147,Nisibis,37.08,41.22,78.80,24,0,6.93,TR,1603125308,Avesis Hotel & Restaurant
167,167,Richards Bay,-28.78,32.04,73.99,81,0,9.60,ZA,1603125313,BON Hotel Waterfront Richards Bay
290,290,Tiznit Province,29.58,-9.50,79.47,24,0,2.06,MA,1603125342,
321,321,Quelimane,-17.88,36.89,74.64,80,0,9.13,MZ,1603125348,Hotel Elite
414,414,Mīnāb,27.15,57.08,71.60,26,0,4.70,IR,1603125372,Minab Tourism Hotel
434,434,Bosaso,11.28,49.18,79.47,59,0,2.35,SO,1603125378,Nayruus Hotel


In [148]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [150]:
# Add marker layer ontop of heat map
hotel_markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Combine layers on map
fig.add_layer(heat_layer)
fig.add_layer(hotel_markers)

# Save map
plt.savefig("../Images/hotel_map.png")

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>